In [ ]:
from transformers import PegasusTokenizer
from datasets import load_dataset
from transformers import PegasusForConditionalGeneration, TrainingArguments, Trainer

# Load model

model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess_data(examples):
    q = "What is the next action?"
    inputs = [f"Question: {q} Context: {c}" for c in zip(examples["context"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=25)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2_Train_Recognition_Full.csv")
test_dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2_Test_Recognition_Full.csv")
# Apply tokenization

dataset = dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched = True)


In [ ]:


# Training arguments
training_args = TrainingArguments(
    output_dir="./pegasus-finetuned",
    # evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=30,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=test_dataset["train"],
)

# Train the model
trainer.train()


In [ ]:
model.save_pretrained("./pegasus")
tokenizer.save_pretrained("./pegasus")


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
# Load fine-tuned model
model = PegasusForConditionalGeneration.from_pretrained("./Recognition/pegasus").to("cuda")
tokenizer = PegasusTokenizer.from_pretrained("./Recognition/pegasus")

def generate_answer(context=""):
    input_text = context
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    output_ids = model.generate(input_ids)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example
print(generate_answer("palpate landmark, take swab"))

In [ ]:
test_dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2_Test.csv")
test_dataset = test_dataset["train"]
# questions = test_dataset["question"]
context = test_dataset["context"]
labels = test_dataset["answer"]    
print(test_dataset)
print(labels)

In [ ]:
generated_answers = []
for i in range(len(context)):
    # print(questions[i], context[i])
    generated_answers.append(generate_answer(context[i]))
    print(generated_answers[i], labels[i])


In [ ]:
print(generated_answers)
print(labels)

In [ ]:
correct = 0
total = 0
for i in range(len(generated_answers)):
    if(generated_answers[i]==labels[i]):
        correct = correct+1
    total = total+1
print(correct/total)